In [1]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import theano
import xarray as xr

print(f"Running on PyMC3 v{pm.__version__}")

Running on PyMC3 v3.11.5


In [4]:
df = pd.read_excel("./HCP_Data_KDAG_Hackathon.xlsx", parse_dates=['Time_Period'])

In [5]:
df = df.drop(['Speaker_Programs_Attended', 'Vouchers_Dropped'], axis=1)
df = df[['Physician_ID', 'Time_Period', 'Physician_Segment', 'Specialty', 'Sales_Rep_Calls', 'Samples_Dropped', 'Emails_Delivered', 'Brand_Rx', 'Market_Rx']]
df['Specialty'] = df['Specialty'].map({'Dermatologist':'D', 'General Physician':'GP', 'Nurse Practitioner':'NP'})
df['Physician_Segment'] = df['Physician_Segment'].map({'3-Low':'L', '2-Medium':'M', '1-High':'H'})
df.head()

,Physician_ID,Time_Period,Physician_Segment,Specialty,Sales_Rep_Calls,Samples_Dropped,Emails_Delivered,Brand_Rx,Market_Rx
0,axt00001,2019-01-04,L,D,1,0,0,0.0,2.80567
1,axt00001,2019-01-11,L,D,1,0,0,0.0,20.57312
2,axt00001,2019-01-18,L,D,1,0,0,0.0,6.16010
3,axt00001,2019-01-25,L,D,1,5,0,0.0,8.95501
4,axt00001,2019-02-01,L,D,1,0,0,0.0,9.13793


Fitting a model to explain Brand_Rx

Brand_Rx[i, c] = a * Specialty + b * Physician_Segment + c * Sales_Rep_Calls + d * Samples_Dropped + e * Emails_Delivered + f * Market_Rx + g

In [6]:
hcp_idxs, hcps = pd.factorize(df.Physician_ID)
coords = {
    "id": hcps,
    "obs_id": np.arange(len(hcp_idxs)),
}

In [7]:
with pm.Model(coords=coords) as hierarchical_model:
    hcp_idx = pm.Data("hcp_idx", hcp_idxs, dims="obs_id")

    # # Hyperpriors for group nodes
    # mu_a = pm.Normal("mu_a", mu=0.0, sigma=100)
    # sigma_a = pm.HalfNormal("sigma_a", 5.0)

    # mu_b = pm.Normal("mu_b", mu=0.0, sigma=100)
    # sigma_b = pm.HalfNormal("sigma_b", 5.0)

    mu_c = pm.Normal("mu_c", mu=0.0, sigma=100)
    sigma_c = pm.HalfNormal("sigma_c", 5.0)

    mu_d = pm.Normal("mu_d", mu=0.0, sigma=100)
    sigma_d = pm.HalfNormal("sigma_d", 5.0)

    mu_e = pm.Normal("mu_e", mu=0.0, sigma=100)
    sigma_e = pm.HalfNormal("sigma_e", 5.0)

    mu_f = pm.Normal("mu_f", mu=0.0, sigma=100)
    sigma_f = pm.HalfNormal("sigma_f", 5.0)

    mu_g = pm.Normal("mu_g", mu=0.0, sigma=100)
    sigma_g = pm.HalfNormal("sigma_g", 5.0)

    # # specialty of physician
    # a = pm.Normal("a", mu=mu_a, sigma=sigma_a, dims="id")
    # # segment of physician
    # b = pm.Normal("b", mu=mu_b, sigma=sigma_b, dims="id")
    # number of calls made by sales_rep
    c = pm.Normal("c", mu=mu_c, sigma=sigma_c, dims="id") # ROI of sales_rep_calls
    # number of samples dropped
    d = pm.Normal("d", mu=mu_d, sigma=sigma_d, dims="id") # ROI of samples_dropped
    # number of emails delivered
    e = pm.Normal("e", mu=mu_e, sigma=sigma_e, dims="id") # ROI of emails_delivered
    # how likely physician is to prescribe other drugs of similar class
    f = pm.Normal("f", mu=mu_f, sigma=sigma_f, dims="id")
    # intercept for each physician, distributed around group mean mu_g
    g = pm.Normal("g", mu=mu_g, sigma=sigma_g, dims="id")

    # Model error
    eps = pm.HalfCauchy("eps", 5.0)

    # Expected value of outcome
    brand_rx_est = c[hcp_idx] * df.Sales_Rep_Calls.values + d[hcp_idx] * df.Samples_Dropped.values + \
                    e[hcp_idx] * df.Emails_Delivered.values + f[hcp_idx] * df.Market_Rx.values + g[hcp_idx]

    # Data likelihood
    brand_rx_like = pm.Normal("brand_rx_like", mu=brand_rx_est, sigma=eps, observed=df.Brand_Rx.values, dims="obs_id")


In [10]:
with hierarchical_model:
    hierarchical_trace = pm.sample(target_accept=0.95, random_seed=42, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [eps, g, f, e, d, c, sigma_g, mu_g, sigma_f, mu_f, sigma_e, mu_e, sigma_d, mu_d, sigma_c, mu_c]


ValueError: Not enough samples to build a trace.

In [ ]:
import pickle

with open('model.pickle', 'wb') as f:
    pickle.dump(hierarchical_model, f)
    
with open('hierarchical_trace.pickle', 'wb') as f:
    pickle.dump(hierarchical_trace, f)